# Python programs

## Handling command-line arguments

With command-line scripts it's often useful to get parameters from the command line.

All of the arguments sent to the script are available in the `sys.argv`:

```python
import sys

def main():
    print("This is a CLI tool")
    print(sys.argv)

main()
```

| EXAMPLE: |
| :------- |
| See [01: command-line args](mini-projects/01_command-line-args/README.md) for a runnable example. |

## Executing code only as main script

The following structure is used to identify a portion of the code that should be executed only when the file is run as main script and not when imported as a module:

```python
if __name__ == "__main__":
    main()
else:
    # module specific initialization code if any
```

If a file with this structure is called as a script, the variable `__name__` will be set to `"__main__"`, which means the controlling function `main()` will be invoked.

By contrast, if the script has been imported as a module by some other script, its name will be its filename, and the code won't be executed. Note that you can include an *else* block that will only be executed if the file has been imported as a module.

## Redirecting the input and output of a script

In Linux, it's common to redirect the contents of a file or the output of another process intro a script instead of using keyboard output, or to redirect the output of a script to the input of another process.

To redirect a file into input instead of having the user enter input you need to specify that you want to read from stdin. In Python the standard input (stdin) is access through `sys.stdin`.

Similarly, you can write to the standard output (stdout) using `sys.stdout`. Note that `print` writes to stdout by default.

To redirect from a file to input you would use `<` on the command line, and for sending the output to file you'd use `>`.

| EXAMPLE: |
| :------- |
| See [02: stdin/stdout](mini-projects/02_redirect-stdin-stdout/README.md) for a runnable example. |

The script in the example above can receive an infile as stdin using the syntax:

```bash
python script.py me I < infile
```

And you can redirect the std output to file doing:

```bash
python script.py me I < infile > outfile
```

You can also use the `|` command to pipe the output of one command as the input of another command.

```bash
python script.py 0 zero < infile | python script.py 1 one > outfile
```

The output of the process corresponding to `python script.py 0 zero < infile` is directed to the input of the second run of the script, which ultimately redirects the output to a file named `outfile`.

## The `argparse` module

You can configure a script to accept command-line options as well as arguments.

The `argparse` module provides support for parsing different types of arguments and can even generate usage messages.

Consider the following program:

```python
from argparse import ArgumentParser

def main():
    parser = ArgumentParser()
    parser.add_argument("indent", type=int, help="indent value for the report")
    parser.add_argument("input_file", help="file from where data is read from")

    # Optional arguments
    parser.add_argument("-f", "--file", dest="filename", help="file where report is written to")
    parser.add_argument("-x", "--xray", help="specify xray strength factor")
    parser.add_argument("-q", "--quiet", action="store_false", dest="verbose", default=True, help="don't print status messages to stdout")

    args = parser.parse_args()

    print("arguments:", args)

if __name__ == '__main__':
    main()
```

This code creates an instance of `ArgumentParser` and then adds two positional arguments, `indent` and `input_file`. Those are the arguments entered after all the optional arguments have been parsed.

By convention, positional arguments are those without a prefix character (usually `"-"`). Those are required and might have a specific type, like `indent` which must be a parseable `int`.

Then comes the optional arguments like `-f` or `--file`. As discussed, optional arguments begin with the prefix character `"-"`.

The final argument is a bit convoluted:
```python
parser.add_argument("-q", "--quiet", action="store_false", dest="verbose", default=True, help="don't print status messages to stdout")
```
 
That snippet declares the `-q`/`--quiet` optional argument with a default value of `True` and destination variable named `verbose`, which is the opposite of quiet. Thus, when not given, the default value will be `True`, that is`verbose=True`. Conversely, if given, `store_false` will make `verbose=False`.

The `argparse` module returns a Namespace object containing the arguments as attributes.

The values of the arguments can be accessed using the dot notation. If there's no argument for an option, its value will be `None`.

| EXAMPLE: |
| :------- |
| See [03: argparse](mini-projects/03_argparse-command-line-options/README.md) for a runnable example. |

## Using the `fileinput` module

The `fileinput` module provides support for processing lines of input from one or more files. 

It automatically reads the command-line arguments (out of `sys.argv`), takes them as a list of input files, opens them, and starts serving them to your script using a simple iterator:

```python
for line in fileinput.input():
    print(line, end="")
```

Note that if no command-line arguments are present, the stdin will be captured.

The `fileinput` module provides several other functions:

+ get the total number of lines that have been read: `lineno`
+ the number of lines that have been read out of the current file: `filelineno`
+ the name of the current file: `filename`
+ whether this is the first line of a file: `isfirstline`
+ whether the stdin is currently being read: `isstdin`
+ skip to the next file: `nextfile`
+ close the whole stream: `close`

You can call `fileinput.input` with a single filename or a list of filenames and they'll be used as its input files instead of `sys.argv`.

To read from a single file:

```python
for line in fileinput.input("infile_1.txt"):
    ...
```

To read from multiple files:

```python
for line in fileinput.input("infile_1.txt"):
    ...
```



There are additional advanced capabilities, such as the `inplace` option that leaves its output in the same file as its input while leaving the original around as a backup file.


| EXAMPLE: |
| :------- |
| See [04: fileinput](mini-projects/04_fileinput/README.md) for a runnable example. |

## Making the script directly executable in Unix

If you're developing the script to be used in the OS, you can add the following shebang:

```bash
#! /usr/bin/env python3
```

Note that it will be executed using the system's python (instead of the Python configured in your virtual environment), so you have to make sure that all the required libraries are available.

## Programs and modules

For small scripts, a single function works well, but if the script grows separating the controlling function from the rest of the code makes the program easier to read and maintain.

Consider the following script that takes as an argument a number and returns its English-language name:

```python
import sys

_1_to_9_dict = {
    "0": "",
    "1": "one",
    "2": "two",
    "3": "three",
    "4": "four",
    "5": "five",
    "6": "six",
    "7": "seven",
    "8": "eight",
    "9": "nine"
}

_10_to_19_dict = {
    "0": "ten",
    "1": "eleven",
    "2": "twelve",
    "3": "thirteen",
    "4": "fourteen",
    "5": "fifteen",
    "6": "sixteen",
    "7": "seventeen",
    "8": "eighteen",
    "9": "nineteen"
}

_20_to_90_dict = {
    "2": "twenty",
    "3": "thirty",
    "4": "forty",
    "5": "fifty",
    "6": "sixty",
    "7": "seventy",
    "8": "eighty",
    "9": "ninety"
}

def num_2_words(num_str: str):
    if num_str == "0":
        return "zero"
    if len(num_str) > 2:
        return "Number must be between 0 and 99"
    num_str = "0" + num_str
    tens, ones = num_str[-2], num_str[-1]
    if tens == "0":
        return _1_to_9_dict[ones]
    if tens == "1":
        return _10_to_19_dict[ones]
    else:
        return _20_to_90_dict[tens] + " " + _1_to_9_dict[ones]

def main():
    print(num_2_words(sys.argv[1]))

if __name__ == "__main__":
    main()
```

It's standard to have the call to the controlling function (in this case `main()`) at the bottom. 

The controlling function can be either defined at the top or right above where it's called (preferred).

This approach separates the plumbing (the `main()` definition and its invocation) from the code that does the real work.

It's quite common to find functions in Python programs that can be useful outside of the script. In the script above, `num_2_words()` would be such function.

The script translates a number to its English-language name by taking the number from the command-line arguments, but the `num_2_words()` function would be helpful in many other scenarios.

| EXAMPLE: |
| :------- |
| See [05: structuring a large script](mini-projects/05_large-script/README.md) for a runnable example. |

## Distributing Python applications

The current standard way of packaging and distributing Python modules and applications is to use packages called wheels.

Wheels are designed to make installing Python code more reliable and to help manage dependencies.

However, if you have an application that's in multiple modules, you can also distribute it as an executable zip file. This format relies on two facts about Python:

1. If a zip file contains a file named `__main__.py`, Python can use that file as the entry point to the archive and execute the `__main__.py` file directly. In addition, the zip file's contents will be added to `sys.path`, so they are available to be imported and executed by `__main__.py`.

2. Zip files allow arbitrary contents to be added to the beginning of the archive. If you add a shebang line pointing to a Python interpreter and give the file the needed permissions, the file can become self-contained and executable.

For more information on this subject, please review [zipapp](https://docs.python.org/3/library/zipapp.html) library.

There are additional tools such as [py2exe](https://www.py2exe.org/) and [py2app](https://py2app.readthedocs.io/en/latest/) that can create standalone apps for both Windows and MacOS.